In [1]:
#imports

using JuMP, Gurobi, GLPK, DelimitedFiles, Random

## Relaxação Lagrangiana

In [2]:
function atualize_lowerBound(A,c,x_res,indCost)
    
    (m,n) = size(A)
    I = (1:m)
    J = (1:n)
    x_resLower = zeros(n,1)
    global x_res_temp = zeros(n,1)

    for indice_j in indCost
        indice_j = floor(Int,indice_j)
        if x_res[indice_j] == 1
            x_res_temp[indice_j] = 1
            value_mult = A*x_res_temp 

            flag = 0
            for i in I
                if value_mult[i] > 1

                    flag = 1
                    #break
                end
            end

            if flag == 0
                x_resLower[indice_j] = 1
            end

            global x_res_temp = copy(x_resLower)

        end
    end

    return c'*x_resLower

end

atualize_lowerBound (generic function with 1 method)

In [3]:
function RelaxLagrang(A, c, maxiter = 50, tol = 1e-1)
    spp   = Model(Gurobi.Optimizer);
    (m,n) = size(A)
    
    I = (1:m)
    J = (1:n)
    
    @variable(spp, x[1:n], Bin);
    u     = 1*ones(m,1);  
    zs = []
    irep    = 0;
    iter    = 0;
    lower_b = 0;
    upper_b = Inf;
    
    step_size = 0
    grad      = zeros(m,1);
    indCost   = zeros(1,n)
    
    for j1 in J
        maior_valor = -Inf
        for j2 in J
            if (c[j2] >=  maior_valor) && ((j2 in indCost) == false)
                global maior_indice = j2
                 maior_valor = c[j2]
            end
        end
        indCost[j1] = maior_indice
    end 
    
    
    while iter <= maxiter && abs(upper_b - lower_b) >= tol
        
        iter += 1
        u = max.(0, u - step_size*grad)
        
        @objective(spp, Max, sum(c.*x) - sum(u[i]* (sum(A[i,j]*x[j] for j in (1:n)) - 1) for i in (1:m)))
        optimize!(spp)
        
        x_res    = value.(x)
        soma_res = A*x_res
        z        = objective_value(spp);
        append!(zs, z)
        
        best_ub_old = upper_b;
        upper_b = min(upper_b, z);
        
        if best_ub_old == upper_b
            irep += 1;
        else
            irep = 0;
            lower_b = atualize_lowerBound(A,c,x_res,indCost)[1];

        end
        
        grad = ones(m,1) - soma_res + ones(m,1)*rand(1)/10
        sum_g2 = sum(grad[i]^2 for i in I)
        #step_size = (z - lower_b)/sum_g2
        step_size = 1/iter
        #step_size  = 0.008
        
    end
    
    return zs, iter
end

RelaxLagrang (generic function with 3 methods)

## Load SPP from .dat

In [4]:
function loadSPP(fname)
    f=open(fname)
   
    m, n = parse.(Int, split(readline(f)))
    C = parse.(Int, split(readline(f)))
    A=zeros(Int, m, n)
    for i=1:m
        readline(f)
        for valor in split(readline(f))
          j = parse(Int, valor)
          A[i,j]=1
        end
    end
    close(f)
    return C, A
end

loadSPP (generic function with 1 method)

## Set SPP

In [5]:
function setSPP(solverSelected, C, A)
  m, n = size(A)
  ip = Model(solverSelected)
  @variable(ip, x[1:n], Bin)
  @objective(ip, Max, sum(C.*x))
  @constraint(ip , cte[i=1:m], sum(A[i,j] * x[j] for j=1:n) <= 1)
  return ip, x
end

function setSPP_relaxlin(solverSelected, C, A)
  m, n = size(A)
  ip = Model(solverSelected)
  @variable(ip, 0 <= x[1:n] <= 1)
  @objective(ip, Max, sum(C.*x))
  @constraint(ip , cte[i=1:m], sum(A[i,j] * x[j] for j=1:n) <= 1)
  return ip, x
end

setSPP_relaxlin (generic function with 1 method)

## Branch and Bound

#### Minha versão:

In [6]:
function integer_test(x)
    if floor(x)^2 - x^2 == 0
        return 1
    end
    return 0
end

integer_test (generic function with 1 method)

In [7]:
function branch_ceil(solverSelected, C, A, idxs = [])
  m, n = size(A)
  ip = Model(solverSelected)
  @variable(ip, 0 <= x[1:n] <= 1)
  for idx in idxs
    @constraint(ip, x[idx] == 1)
  end
  @objective(ip, Max, sum(C.*x))
  @constraint(ip , cte[i=1:m], sum(A[i,j] * x[j] for j=1:n) <= 1)
  return ip, x
end

function branch_floor(solverSelected, C, A, idxs = [])
  m, n = size(A)
  ip = Model(solverSelected)
  @variable(ip, 0 <= x[1:n] <= 1)
  for idx in idxs
    @constraint(ip, x[idx] == 0)
  end
  @objective(ip, Max, sum(C.*x))
  @constraint(ip , cte[i=1:m], sum(A[i,j] * x[j] for j=1:n) <= 1)
  return ip, x
end

branch_floor (generic function with 2 methods)

In [8]:
function BranchNBound(A, c, maxiter = 50, tol = 1e-1)
    
    rp, rp_x = setSPP_relaxlin(Gurobi.Optimizer, c, A)
    nodes    = [[rp,[]]]
    s,       = size(nodes)
    values   = []
    idxs     = []
    
    while s >= 0
    
        
        if s == 0
            break
        end
        
        p,idxs   = nodes[1]
        
        if s == 1
            nodes = []
            s     = 0
        else
            nodes = nodes[2:s]
            s,     = size(nodes)
        end

        
        try
           optimize!(p) 
        catch  
           println(" Modelo Infeasible. Cortem as árvores! ")
           ss,       = size(idxs)
           idxs      = idxs[1:ss-1]
            
        end
        append!(values, objective_value(p))
        
        for (i,xs) in enumerate(value.((all_variables(p))))
            if integer_test(xs) == 0
                
                push!(idxs, i)
                print("----------> ", idxs, " <---------")
                print(" BRANCHING ")
                
                floor_prob, f_x = branch_floor(Gurobi.Optimizer, c, A,idxs)
                push!(nodes, floor_prob)
                
                ceil_prob, c_x  =  branch_ceil(Gurobi.Optimizer, c, A,idxs)
                push!(nodes, ceil_prob)
                
                s += 2
                print("--> ", values, " <---")
                break
            end
        end
    end

    return objective_value(rp), value.(rp_x), values
end

BranchNBound (generic function with 3 methods)

### Versão chamando o Gurobi

In [9]:
function Branch_Bound(A,c)
    
    rp, rp_x = setSPP(Gurobi.Optimizer, c, A)

    println("Solving..."); 
    set_optimizer_attribute(rp, "Presolve", 0)
    set_optimizer_attribute(rp, "Cuts", 0)
    set_optimizer_attribute(rp, "Heuristics", 0)
    optimize!(rp) 
end

Branch_Bound (generic function with 1 method)

## Branch and Cut

In [10]:
function Branch_Cut(A,c)
    
    p, p_x = setSPP(Gurobi.Optimizer, c, A)

    println("Solving..."); 
    set_optimizer_attribute(p, "Presolve", 0)
    set_optimizer_attribute(p, "Cuts", 0)
    set_optimizer_attribute(p, "Heuristics", 0)
    set_optimizer_attribute(p, "CliqueCuts", 1) 
    optimize!(p) 
    
end

Branch_Cut (generic function with 1 method)

# Instâncias

In [11]:
A = [1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,
     0,0,0,1,1,0,1,1,0,0,0,0,1,1,1,1,0,1,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,
     0,0,0,0,0,0,1,1,1,1,1,1];

A = reshape(A, 12,7);
c = [1,1,1,1,1,1,1];

In [12]:
RelaxLagrang(A,c)

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 0 rows, 7 columns and 0 nonzeros
Model fingerprint: 0x31521159
Variable types: 0 continuous, 7 integer (7 binary)
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [1e+00, 6e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective 12.0000000
Presolve removed 0 rows and 7 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 12 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.200000000000e+01, best bound 1.200000000000e+01, gap 0.0000%
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 0 rows, 7 columns and 0 nonzeros
Model fingerprint: 0xed12b246
Variable types: 0 co

(Any[12.0, 7.0, 6.883769617706012, 4.285549720090757, 4.004464163024389, 4.074013433633763, 4.0], 7)

In [13]:
Branch_Bound(A,c)

Academic license - for non-commercial use only
Solving...
Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 12 rows, 7 columns and 26 nonzeros
Model fingerprint: 0xdba4e680
Variable types: 0 continuous, 7 integer (7 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Variable types: 0 continuous, 7 integer (7 binary)

Root relaxation: objective 4.000000e+00, 0 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0       4.0000000    4.00000  0.00%     -    0s

Explored 0 nodes (0 simplex iterations) in 0.00 seconds
Thread count was 8 (of 8 available processors)

Solution count 1: 4 

Optimal solution found (tolerance 1.00e-04)
Best objective 4.0000000

In [14]:
Branch_Cut(A,c)

Academic license - for non-commercial use only
Solving...
Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 12 rows, 7 columns and 26 nonzeros
Model fingerprint: 0xdba4e680
Variable types: 0 continuous, 7 integer (7 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Variable types: 0 continuous, 7 integer (7 binary)

Root relaxation: objective 4.000000e+00, 0 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0       4.0000000    4.00000  0.00%     -    0s

Explored 0 nodes (0 simplex iterations) in 0.00 seconds
Thread count was 8 (of 8 available processors)

Solution count 1: 4 

Optimal solution found (tolerance 1.00e-04)
Best objective 4.0000000

## Instâncias Maiores

In [15]:
fname = "Instances\\Data\\pb_100rnd0100.dat"; 
c, A = loadSPP(fname);

In [16]:
Branch_Cut(A,c)

Academic license - for non-commercial use only
Solving...
Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 500 rows, 100 columns and 1000 nonzeros
Model fingerprint: 0x3c1811da
Variable types: 0 continuous, 100 integer (100 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Variable types: 0 continuous, 100 integer (100 binary)

Root relaxation: objective 5.145000e+02, 63 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  514.50000    0   97          -  514.50000      -     -    0s
     0     0  406.60000    0   90          -  406.60000      -     -    0s
     0     0  406.14286    0   90          -  406.14286      -     -    0s
     0     0  406.14286    0   

In [17]:
Branch_Bound(A,c)

Academic license - for non-commercial use only
Solving...
Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 500 rows, 100 columns and 1000 nonzeros
Model fingerprint: 0x3c1811da
Variable types: 0 continuous, 100 integer (100 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Variable types: 0 continuous, 100 integer (100 binary)

Root relaxation: objective 5.145000e+02, 63 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  514.50000    0   97          -  514.50000      -     -    0s
     0     0  514.50000    0   97          -  514.50000      -     -    0s
     0     2  514.50000    0   97          -  514.50000      -     -    0s
*   97   106              18   

In [18]:
fname = "Instances\\Data\\pb_200rnd0100.dat"; 
c, A = loadSPP(fname);

In [19]:
Branch_Bound(A,c)

Academic license - for non-commercial use only
Solving...
Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 1000 rows, 200 columns and 2978 nonzeros
Model fingerprint: 0x8f82369d
Variable types: 0 continuous, 200 integer (200 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Variable types: 0 continuous, 200 integer (200 binary)

Root relaxation: objective 5.810675e+02, 720 iterations, 0.05 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  581.06750    0  176          -  581.06750      -     -    0s
     0     0  581.06750    0  176          -  581.06750      -     -    0s
     0     2  581.06750    0  176          -  581.06750      -     -    0s
*  173   173              37 

In [20]:
Branch_Cut(A,c)

Academic license - for non-commercial use only
Solving...
Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 1000 rows, 200 columns and 2978 nonzeros
Model fingerprint: 0x8f82369d
Variable types: 0 continuous, 200 integer (200 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Variable types: 0 continuous, 200 integer (200 binary)

Root relaxation: objective 5.810675e+02, 720 iterations, 0.07 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  581.06750    0  176          -  581.06750      -     -    0s
     0     0  536.71608    0  178          -  536.71608      -     -    0s
     0     0  535.90817    0  177          -  535.90817      -     -    0s
     0     0  535.55596    0 

In [21]:
fname = "Instances\\Data\\pb_500rnd0100.dat"; 
c, A = loadSPP(fname);

In [ ]:
Branch_Cut(A,c)

## Instâncias Aleatórias 

In [ ]:
rng = MersenneTwister(10000)
# 
A = bitrand(rng,(25,25))
c = 100*rand(25,1)
c = floor.(c); #.+ 1;

In [ ]:
Branch_Bound(A,c)

In [ ]:
Branch_Cut(A,c)